# This notebook is part 2 of using autogen with local ai models to query an arbitrary database.
In this notebook I will connect a local AI model to a local database and query it.  I will not be using autogen. This will be the same as example 1 but instead of chatGPT I will use a local model and I will run from windows instead of linux.
I'm using this reference https://github.com/disler/multi-agent-postgres-data-analytics/tree/v1-prompt-engineering-an-entire-codebase

## Install ODBC Driver for MSSQL
For this example I did not need to install any additional ODBC drivers in windows.

In [2]:
%pip install pyodbc
%pip install python-dotenv
%pip install --upgrade --force-reinstall openai==0.28.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached frozenlist-1.4.0-cp310-cp310-manylinux_2_5_x86_64.many

## Create code to handle connecting to the database
The settings for the database should obviously be in a config file, but for now I'm just going to hard code them in.

In [3]:
# connect to MSSQL database
import pyodbc

SERVER = '127.0.0.1'
DATABASE = 'TimeBasedCommitments'
USERNAME = 'sa'
PASSWORD = 'BadDefaultPassword!'

connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

conn = pyodbc.connect(connectionString)

# conn.close()

# Get all table definitions
The table information will be passed to the AI prompt so it can create queries.

In [4]:
# Get table names
get_all_tables_stmt = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES;"
cursor = conn.cursor()
cursor.execute(get_all_tables_stmt)
table_names = [row[0] for row in cursor.fetchall()]

# Get all table definitions and format them into CREATE TABLE statements
get_def_stmt = f"""
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = ?
"""

definitions = []
for table_name in table_names:
    cursor.execute(get_def_stmt, (table_name,))
    rows = cursor.fetchall()

    create_table_stmt = f"CREATE TABLE {table_name} (\n"
    for row in rows:
        create_table_stmt += f"{row[0]} {row[1]},\n"
    create_table_stmt = create_table_stmt.rstrip(",\n") + "\n);"
    
    definitions.append(create_table_stmt)

table_definitions = "\n\n".join(definitions)

## Setup the AI model
Sets api base to point at local model instead of ChatGPT

In [12]:
import openai
import os
import dotenv

dotenv.load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")

## Build the prompt
I tried using the same prompt as GPT4 but it wasnt working because llama2 wasnt respecting the response format.  Each time it would put the query in a different place and wasnt using the delimiter.  Instead I made the request simpler and ask for ONLY the SQL query.  This may be fixable when I am using autogen and the llm models can deal with eachothers responses to pull out the relevant data.

In [ ]:
# This is the user input
user_question = "How many users are there in the database?"

# The rest of this prompt building is back end logic
## Build the request
prompt = f"Create a SQL query that can be used to answer this question: {user_question}. "
POSTGRES_TABLE_DEFINITIONS_CAP_REF = "TABLE_DEFINITIONS"
prompt_suffix = f"Use these {POSTGRES_TABLE_DEFINITIONS_CAP_REF} to satisfy the database query."

prompt = f"""{prompt} {prompt_suffix}\n\n{POSTGRES_TABLE_DEFINITIONS_CAP_REF}\n\n{table_definitions}"""

## Append the response format
prompt_suffix = f"\n\nRespond only with the SQL query. I need to be able to easily parse the sql query from your response."
cap_ref_content = f"""RESPONSE_FORMAT
---------
<sql query exclusively as raw text>"""
prompt = f"""{prompt} {prompt_suffix}\n\n{cap_ref_content}"""

print("\n\n-------- PROMPT --------")
print(prompt)

## Send the prompt to the AI model
NOTE: I had to pin the openai version to 0.28 becuase the 1.0 version has breaking changes.  To keep this example short I did not want to go through the migration guide.
https://github.com/openai/openai-python/discussions/742

In [26]:
model = "llama2"
temperatures = [0.1]
max_tokens = 1024

response = openai.ChatCompletion.create(
    model=model,
    temperatures=temperatures,
    max_tokens=max_tokens,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

print("\n\n-------- RESPONSE --------")
print(response)



-------- RESPONSE --------
{
  "id": "chatcmpl-1",
  "object": "chat.completion",
  "created": 1,
  "model": "koboldcpp/llama-2-13b-chat.Q5_K_M",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\nSELECT COUNT(*) FROM tblUsers;"
      },
      "finish_reason": "length"
    }
  ]
}


## Format the response and extract the SQL query

In [27]:
# This was taken directly from the example.  I need to review it later.
def safe_get(data, dot_chained_keys):
    """
    {'a': {'b': [{'c': 1}]}}
    safe_get(data, 'a.b.0.c') -> 1
    """
    keys = dot_chained_keys.split(".")
    for key in keys:
        try:
            if isinstance(data, list):
                data = data[int(key)]
            else:
                data = data[key]
        except (KeyError, TypeError, IndexError):
            return None
    return data

prompt_response = safe_get(response, "choices.0.message.content")

sql_query = prompt_response.strip()

print(f"\n\n-------- PARSED SQL QUERY --------")
print(sql_query)



-------- PARSED SQL QUERY --------
SELECT COUNT(*) FROM tblUsers;


## Execute the query and print the results
Also make sure to close the database connection here!

In [28]:
cursor = conn.cursor()
cursor.execute(sql_query)
rows = cursor.fetchall()
print(f"\n\n-------- SQL QUERY RESULTS --------")
print(rows)
conn.close()



-------- SQL QUERY RESULTS --------
[(150,)]
